# Optimization Homework 4
1. A cylindrical tank has its height $=h$ and the radius of the top and bottom $=r$ (both in meters). What $h$ and $r$ values will minimize the total surface area (including the top, bottom, and the side) of this cylindrical tank. We are also told that the volume of this tank has to be fixed (i.e. $\pi r^2h= 20 m^3)$. Please reformulate the objective function to include a penalty term, and solve the problem numerically using any method we have learned (e.g. the Nelder-Mead method or DFP method).

The surface area for a cylinder is $f(h, r) = 2\pi r(h+r)$, and we are given the constraint $g(h, r) = \pi r^2h = 20$. 
We can reformulate the objective function to an *interior* penalty method as
$$\varphi(h,r, r_k) = f(h, r) + r_kg(h,r)^2 = 2\pi r(h+r) + r_k (\pi r^2h-20)^2$$

In [74]:
import math
import numpy as np

# Objective function with penalty term
def obj(point, rk=1):
    h, r = point[0], point[1]
    first = 2 * math.pi * r * (h + r)
    sec = rk * math.pow((math.pi * (r ** 2) * h - 20), 2)
    return first + sec
    

## Nelder Mead
**Note**: In the book, it's suggested that for equality constraints, we have $H(r_k) = \frac{1}{\sqrt{r_k}}$, whereas in the class notes we just use $r_k$. I think both should be ok as long as we choose $r_k$ well each iteration.

In [81]:
tol = .001
alpha, beta, gamma = 1, 2, 0.5

# Initial simplex
simp = np.array([[0, 0],
                 [10, 0],
                 [-10, -10]
                ])
while True:
    # Get the minimum and maximum points
    minval, maxval = obj(simp[0,:]), obj(simp[0,:])
    vals = np.apply_along_axis(obj, 1, simp)
    
    xmin = simp[np.argmin(vals)]
    xmax = simp[np.argmax(vals)]
    xmed = simp[np.argsort(vals)[1]]
    if np.linalg.norm(xmin - xmax) < tol:
        break
        
    xa = (xmin + xmed ) / 2
    
    xr = xa + alpha*(xa - xmax)
    
    if obj(xmin) > obj(xr):
        # Expansion point
        xe = xa + gamma * (xr - xa)
        xmax = xe if obj(xr) > obj(xe) else xr
        simp = np.array([xmin, xmed, xmax])
        continue
    if obj(xmed) > obj(xr):
        xmax = xr
        simp = np.array([xmin, xmed, xmax])
        continue
    xp = xr if obj(xr) < obj(xmax) else xmax
    xc = xa + beta * (xp - xa)
    if obj(xc) > obj(xp):
        simp = simp + (xmin - simp) / 2
        continue
    else:
        simp = np.array([xmin, xmed, xc])
    
print(f'Approximate optimal point: {np.mean(simp, axis=0)}')

Approximate optimal point: [2.90791829 1.45395915]


## 2
Solve the following muscle force distribution problem. The $F_i$ values are to be solved. Values of $A_i, M_i$, and $d_i$ are given below.
Minimize
$$Z=\sum_{i=1}^{9}(\frac{F_i}{A_i})^n, n = 2$$

subject to
$$f_1 = d_1F_1-d_2F_2-d_{3a}F_3 - M_1=0$$
$$f_2 = -d_{3k}F_3 + d_4F_4 + d_{5k}F_5 -d_6F_6 - d_{7k}F_7 -M_2 = 0$$
$$f_3 = d_{5b}F_5 -d_{7b}F_{7}+d_8F_8 -d_9F_9-M_3=0$$
$$F_i\geq0 (i=1,2,\dots, 9)$$

The values of the constants are gien below:
$$M_1 ==4, M_2 = 33, M_3=31$$
The values of $d_1, d_2, d_{3a}, d_4, d_{5k}, d_{5h}, d_6, d_{7k}, d_{7h}, d_8, d_9$ are included in the following vector: $d=[0.0298 ,0.044 ,0.044 0.0138 ,0.0329 ,0.0329, 0.0279, 0.025 ,0.025 ,0.0619 ,0.0317 ,0.0368]$.
The values of $A_i$ are also included in the vector $A=[11.5, 92.5, 44.3 98.1, 20.1, 6.1, 45.5, 31.0, 44.3]$